#  Введение в git


Что происходит в наш с вами типичный рабочий день:

- Добавление новых фич
> Как это сделать? Можно изменять непосредственно рабочий проект, набажить, забыть, как было раньше, все испортить. Можно скопировать проект (не забыть осознанно назвать папку), добавить фичу в копии, потом мучительно "слиять", потому что кто-то уже изменил рабочий проект, а это затронуло вещи, используемые новой фичей. Каша, неразбериха.
- Фикс багов
> Копируем проект, фиксим баг, сливаем обратно. Все это с теми же проблемами.
- Развитие альтернативных версий проекта
> Копируем проект, долго разрабатываем. При слиянии еще больше кошмаров.


Все эти кейсы становятся много менее кошмарными при использовании git. В этом ноутбуке поговим о его базовой структуре и командах.



## Базовое устройство

Здесь будет описано устройство репозитория, а именно что, где и как все хранится, это будет нужно для понимания, что на самом деле делают все команды.

#### На каждой машине хранится копия всего репозитория.


#### В самом репозитории хранятся снапшоты -- "слепки" (сжатые копии) всей файловой системы в каком-то состоянии проекта.

- В каждом снапшоте хранятся копии(!) рабочей директории (а не diff-ы) 
- Если в новом снапшоте файл остался неизменным, то будет хранится ссылка на файл в предыдущем слепке.

![snapshots](https://git-scm.com/book/en/v2/images/snapshots.png)

#### Снапшоты индексируются коммитами

- Коммит - объект, содержащий ссылку на слепок, информацию об авторе коммита, комментарий, а также ссылки на предыдущий коммит (или несколько ссылок, если этот коммит -- результат слияния веток).

![commits_snapshots](https://git-scm.com/book/en/v2/images/commits-and-parents.png)


#### Ветка -- это указатель на коммит.

- На машине она представляет собой лишь файл, содержащий чек-сумму коммита
- На текущую (рабочую) ветку указывает указатель HEAD
- Переключение между ветками пердставляет собой просто перенос указателя.

![branch](https://git-scm.com/book/en/v2/images/advance-master.png)

---

## Основные принципы

Осознанавание базовых принципов работы с файлами проекта, отслеживаемого git'ом, позволит избежать большинства ошибок и сэкономит много времени.


#### Файлы в репозитории могут находиться в трех различных состояниях: commited, modified, staged.

- Commited - файлы сохранены в локальной БД
- Modified - файлы изменены, но изменения не внесены в локальную БД
- Staged - файлы помечены, будут включены в следующий коммит



#### Соответственно, в гите есть три разных "пространства":  repository, working directory, staging area 
 
- Repository -- Хранятся метаданные и БД репозитория.
- Working directory -- локальная копия рабочей версии проекта
- Staging area (или index) -- файлы, которые войдут в следующий коммит

![states](https://git-scm.com/book/en/v2/images/areas.png)

#### Рабочий процесс


- Меняем файлы в working directory
- Указываем в staging area, какие именно изменения будут учтены при следующем коммите
- Делаем коммит (записываем снапшот в БД и создаем коммит-объект)

![](https://git-scm.com/book/en/v2/images/lifecycle.png)

---


# Базовые команды


## Создание репозитория





### Создание локального репозитория

#### git init

git init folder

Внутри folder создается папка .git -- сам репозиторий.

In [1]:
!if not exist folder mkdir folder
!git init folder
!dir folder\.git

Initialized empty Git repository in C:/Users/Roman Khudorozhkov/Documents/az_training/git_tutorial/folder/.git/
 Volume in drive C is OS
 Volume Serial Number is 2CFE-F62B

 Directory of C:\Users\Roman Khudorozhkov\Documents\az_training\git_tutorial\folder\.git

28.11.2017  15:31               112 config
28.11.2017  15:31                73 description
28.11.2017  15:31                23 HEAD
28.11.2017  15:31    <DIR>          hooks
28.11.2017  15:31    <DIR>          info
28.11.2017  15:31    <DIR>          objects
28.11.2017  15:31    <DIR>          refs
               3 File(s)            208 bytes
               4 Dir(s)  138Â 259Â 832Â 832 bytes free


Указатель HEAD:

In [2]:
!type folder\.git\HEAD

ref: refs/heads/master



### Клонирование

#### git clone

git clone url folder -- клонирует репозиторий по адресу url в folder, делает чекаут активной ветки проекта.

In [3]:
!git clone folder folder_clone

Cloning into 'folder_clone'...
done.


Флаги:
- -n == --no-checkout -- никуда не ставить HEAD (не делать чекаутов)
- -b == --branch name -- сделать чекаут ветки name

## Работа с репозиторием

### Работа с локальной копией







In [3]:
import os
os.chdir('folder')

## git status

узнать текущий статус репозитория

In [4]:
!git status

On branch master

No commits yet

nothing to commit (create/copy files and use "git add" to track)


Флаги:
- -s == --short короткий вывод (о нем позднее)
- --porcelain -- вывод, удобный для парсинга скриптами
- --long -- длинный вывод (включен по дефолту) 
- -u == --untracked mode -- как показывать неотслеживаемые файлы (no - не показывать)

Сделаем видимость работы....

In [5]:
with open('some_file.txt', 'w') as file:
    file.writelines(['work\n','hard\n'])

In [6]:
!git status

On branch master

No commits yet

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	some_file.txt

nothing added to commit but untracked files present (use "git add" to track)


In [7]:
!git status -u no

On branch master

No commits yet

nothing to commit (create/copy files and use "git add" to track)


In [8]:
!git status -s

?? some_file.txt


In [9]:
!git status --porcelain

?? some_file.txt


В чем разница между двумя последними флагами? Ни в чем, но вывод последней не зависит от настроек пользователя и версий git

## git add

Хотим включить some_file в следующий коммит. git советует воспользоваться add:

In [10]:
!git status

On branch master

No commits yet

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	some_file.txt

nothing added to commit but untracked files present (use "git add" to track)


#### Если только хотим посмотреть на то, что добавим, то сделаем с флагом --dry-run == -n:

In [11]:
!git add -n some_file.txt

add 'some_file.txt'


The file will have its original line endings in your working directory.


In [12]:
!git add some_file.txt

The file will have its original line endings in your working directory.


In [13]:
!git status 

On branch master

No commits yet

Changes to be committed:
  (use "git rm --cached <file>..." to unstage)

	new file:   some_file.txt



In [14]:
!git status -s

A  some_file.txt


Теперь some_file находится в состоянии staged и находится в индексе (index, staging area)

Что будет, если мы опять изменим файл?

In [15]:
with open('some_file.txt', 'a') as file:
    file.write('work harder! \n')

In [16]:
!git status

On branch master

No commits yet

Changes to be committed:
  (use "git rm --cached <file>..." to unstage)

	new file:   some_file.txt

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   some_file.txt



In [17]:
!git status -s

AM some_file.txt


Обратите внимание, что в следующий коммит пойдет только первая версия файла! Заодно разберемся с флагом -s: неотслеживаемые файлы обозначены ??. Левая колонка отвечает за staging area, правая -- за working tree. А == added, M == modified.

In [18]:
!git add some_file.txt

The file will have its original line endings in your working directory.


In [19]:
!git status

On branch master

No commits yet

Changes to be committed:
  (use "git rm --cached <file>..." to unstage)

	new file:   some_file.txt



In [20]:
!git status -s

A  some_file.txt


Теперь в коммит пойдет последняя версия файла

## git commit

Теперь хотим записать изменения в репозиторий. Для этого нужно сделать коммит:

In [21]:
from subprocess import call
call(["git", "commit", '-m', 'added some file']);

In [22]:
!git status

On branch master
nothing to commit, working tree clean


#### Писать add, а потом commit иногда бессмысленно, первый этап можно пропустить:

In [23]:
with open('some_file.txt', 'a') as file:
    file.write('i\'m tired')

In [24]:
!git status -s

 M some_file.txt


In [25]:
call(["git", "commit", '-a', '-m', 'modified some file']);

также флаг -а удалит из индекса все файлы, которые были удалены из рабочей директории

In [26]:
!git status

On branch master
nothing to commit, working tree clean


Если хотим изменить содержание коммита предыдущего сообщения, то набираем

$ git commit --amend

откроется редактор, в котором можно будет все исправить.

Добавление этого флага также учтет все произошедшие изменения в staging area. Это способ "поправить" последний коммит, если не хочется ими "спамить" (на самом деле создастся новый коммит, который заменит предыдущий)

Для dry run нужно добавить флаг --dry-run

## откаты

## unstaging

In [27]:
with open('file.txt', 'a') as file:
    file.write('i\'m useless')

In [28]:
!git add file.txt

In [29]:
!git status

On branch master
Changes to be committed:
  (use "git reset HEAD <file>..." to unstage)

	new file:   file.txt




#### git подсказывает, что если хотим файл "забрать" из staging area (файл вернется в то состояние, в котором он был), нужно набрать

In [30]:
!git reset HEAD file.txt

In [31]:
!git status -s

?? file.txt


In [32]:
with open('file.txt', 'a') as file:
    file.write('i\'m so useless....')

In [33]:
!git status

On branch master
Untracked files:
  (use "git add <file>..." to include in what will be committed)

	file.txt

nothing added to commit but untracked files present (use "git add" to track)


In [34]:
!git add file.txt

In [35]:
!git status 

On branch master
Changes to be committed:
  (use "git reset HEAD <file>..." to unstage)

	new file:   file.txt



In [36]:
call(["git", "commit", '-m', 'added file']);

In [37]:
with open('file.txt', 'a') as file:
    file.write('i\'m so useless....')

In [38]:
!git status

On branch master
Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   file.txt

no changes added to commit (use "git add" and/or "git commit -a")


#### Вернем файл в то состояние, в котором был сделан коммит

In [39]:
!git checkout -- file.txt

In [40]:
!git status

On branch master
nothing to commit, working tree clean


### git rm

Удаляет файл из индекса и рабочей директории

- -r - рекурсивное удаление
- --dry-run

In [41]:
!git rm file.txt

rm 'file.txt'


In [42]:
!git status

On branch master
Changes to be committed:
  (use "git reset HEAD <file>..." to unstage)

	deleted:    file.txt



In [43]:
call(['git', 'commit', '-m', 'deleted file']);

In [44]:
!git status

On branch master
nothing to commit, working tree clean


In [45]:
!dir

 Volume in drive C is OS
 Volume Serial Number is 2CFE-F62B

 Directory of C:\Users\Roman Khudorozhkov\Documents\az_training\git_tutorial\folder

28.11.2017  15:32    <DIR>          .
28.11.2017  15:32    <DIR>          ..
28.11.2017  15:32                36 some_file.txt
               1 File(s)             36 bytes
               2 Dir(s)  138Â 259Â 689Â 472 bytes free


#### Если мы сделали файл staged, а потом передумали, то можно вызвать rm с флагом --cached (удалит из индекса, но не с машины)

In [46]:
with open('file.txt', 'a') as file:
    file.write('i\'m so useless....')
    
!git add file.txt
!git status -s

A  file.txt


In [47]:
!git rm --cached file.txt

rm 'file.txt'


In [48]:
!git status

On branch master
Untracked files:
  (use "git add <file>..." to include in what will be committed)

	file.txt

nothing added to commit but untracked files present (use "git add" to track)


In [49]:
!git add file.txt
call(['git', 'commit', '-m', 'added file']);


#### Что будет, если просто удалить файл из working tree:

In [50]:
!del file.txt

In [51]:
!git status

On branch master
Changes not staged for commit:
  (use "git add/rm <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	deleted:    file.txt

no changes added to commit (use "git add" and/or "git commit -a")


In [52]:
call(['git', 'commit', '-a', '-m', 'deleted file']);

In [53]:
!git status

On branch master
nothing to commit, working tree clean


In [54]:
!dir

 Volume in drive C is OS
 Volume Serial Number is 2CFE-F62B

 Directory of C:\Users\Roman Khudorozhkov\Documents\az_training\git_tutorial\folder

28.11.2017  15:32    <DIR>          .
28.11.2017  15:32    <DIR>          ..
28.11.2017  15:32                36 some_file.txt
               1 File(s)             36 bytes
               2 Dir(s)  138Â 259Â 681Â 280 bytes free


## просмотр изменений

## git diff

In [55]:
!dir

 Volume in drive C is OS
 Volume Serial Number is 2CFE-F62B

 Directory of C:\Users\Roman Khudorozhkov\Documents\az_training\git_tutorial\folder

28.11.2017  15:32    <DIR>          .
28.11.2017  15:32    <DIR>          ..
28.11.2017  15:32                36 some_file.txt
               1 File(s)             36 bytes
               2 Dir(s)  138Â 259Â 681Â 280 bytes free


In [56]:
with open('some_file.txt','a') as file:
    file.write('lalala\n')

In [116]:
!git status -s

 M some_file.txt


In [117]:
!git diff

diff --git a/some_file.txt b/some_file.txt
index c933383..e769960 100644
--- a/some_file.txt
+++ b/some_file.txt
@@ -1,4 +1,4 @@
 work
 hard
 work harder! 
-i'm tired
\ No newline at end of file
+i'm tiredlalala


The file will have its original line endings in your working directory.


In [118]:
!git add some_file.txt

The file will have its original line endings in your working directory.


In [119]:
!git diff


#### чтобы посмотреть разницу между файлами в последнем коммите и staged, нужно передать флаг cached (или staged):

In [124]:
!git diff --staged

diff --git a/some_file.txt b/some_file.txt
index c933383..e769960 100644
--- a/some_file.txt
+++ b/some_file.txt
@@ -1,4 +1,4 @@
 work
 hard
 work harder! 
-i'm tired
\ No newline at end of file
+i'm tiredlalala


#### посмотреть разность staged с другой веткой (подставь вместо HEAD имя ветки):

In [121]:
!git diff HEAD

diff --git a/some_file.txt b/some_file.txt
index c933383..e769960 100644
--- a/some_file.txt
+++ b/some_file.txt
@@ -1,4 +1,4 @@
 work
 hard
 work harder! 
-i'm tired
\ No newline at end of file
+i'm tiredlalala
